In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import requests
import re
import time
import concurrent.futures

from tqdm import tqdm
tqdm.pandas()

In [2]:
df = pd.read_csv(r"dataset\transfers_complete.csv")
df = df[["team_id", "team_name"]]
df = df.drop_duplicates()

for season in ["09/10", "10/11", "11/12", "12/13", "13/14", "14/15", "15/16", "16/17", "17/18", "18/19", "19/20", "20/21", "21/22", "22/23", "23/24"]:
    df[f"{season}"] = None

df

,team_id,team_name,09/10,10/11,11/12,12/13,13/14,14/15,15/16,16/17,17/18,18/19,19/20,20/21,21/22,22/23,23/24
0,985,Manchester United,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
26,31,Liverpool FC,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
57,631,Chelsea FC,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
83,11,Arsenal FC,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
103,29,Everton FC,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78709,1283,FC Emmen,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
78789,385,Fortuna Sittard,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
81013,133,SC Cambuur Leeuwarden,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
81457,724,FC Volendam,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


### SCRAPER

**From list of ids**

In [3]:
def scrape_seasonal_performance(ids: list):

    HEADERS = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
        'cache-control': 'max-age=0',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36 OPR/62.0.3331.116',
    }

    session = requests.session()
    session.headers.update(HEADERS)

    results = {}
    for id in ids:
        id = 985
        champ_url = f"https://www.transfermarkt.it/generic-team/platzierungen/verein/{id}"
    
        resp = session.get(champ_url)
        html = BeautifulSoup(resp.text)

        team_results = {}
        seasons = html.find("table", class_="items").findAll("tr")[1:]
        for season in seasons:
            try:
                outcome = re.findall('<td class="zentriert">'+ "(.*?)" +'</td>', str(season))
                team_results[outcome[0]] = {'tier': outcome[1], 'wins': int(outcome[2]), 'draws': int(outcome[3]), 'loss': int(outcome[4]),
                        'g_scored': int(outcome[5].split(":")[0]), 'g_conc': int(outcome[5].split(":")[1]), 'g_diff': int(outcome[6]),
                        'pts': float(outcome[7].split(":")[0]),
                        'rank': int(outcome[8].replace("<b>", "").replace("</b>", ""))}
            except:
                pass

        results[id] = team_results

    return results

In [119]:
ids = [985, 1390]

print(scrape_seasonal_performance(ids))

{985: {'23/24': {'tier': 'Primo livello', 'wins': 17, 'draws': 6, 'loss': 14, 'g_scored': 55, 'g_conc': 58, 'g_diff': -3, 'pts': 57.0, 'rank': 8}, '22/23': {'tier': 'Primo livello', 'wins': 23, 'draws': 6, 'loss': 9, 'g_scored': 58, 'g_conc': 43, 'g_diff': 15, 'pts': 75.0, 'rank': 3}, '21/22': {'tier': 'Primo livello', 'wins': 16, 'draws': 10, 'loss': 12, 'g_scored': 57, 'g_conc': 57, 'g_diff': 0, 'pts': 58.0, 'rank': 6}, '20/21': {'tier': 'Primo livello', 'wins': 21, 'draws': 11, 'loss': 6, 'g_scored': 73, 'g_conc': 44, 'g_diff': 29, 'pts': 74.0, 'rank': 2}, '19/20': {'tier': 'Primo livello', 'wins': 18, 'draws': 12, 'loss': 8, 'g_scored': 66, 'g_conc': 36, 'g_diff': 30, 'pts': 66.0, 'rank': 3}, '18/19': {'tier': 'Primo livello', 'wins': 19, 'draws': 9, 'loss': 10, 'g_scored': 65, 'g_conc': 54, 'g_diff': 11, 'pts': 66.0, 'rank': 6}, '17/18': {'tier': 'Primo livello', 'wins': 25, 'draws': 6, 'loss': 7, 'g_scored': 68, 'g_conc': 28, 'g_diff': 40, 'pts': 81.0, 'rank': 2}, '16/17': {'tier

**From single id**

In [4]:
def scrape_seasonal_performance(id: int):

    HEADERS = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
        'cache-control': 'max-age=0',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36 OPR/62.0.3331.116',
    }

    session = requests.session()
    session.headers.update(HEADERS)

    results = {}
    
    champ_url = f"https://www.transfermarkt.it/generic-team/platzierungen/verein/{id}"
    resp = session.get(champ_url)
    html = BeautifulSoup(resp.text)

    team_results = {}
    seasons = html.find("table", class_="items").findAll("tr")[1:]
    for season in seasons:
        try:
            outcome = re.findall('<td class="zentriert">'+ "(.*?)" +'</td>', str(season))
            team_results[outcome[0]] = {'tier': outcome[1], 'wins': int(outcome[2]), 'draws': int(outcome[3]), 'loss': int(outcome[4]),
                    'g_scored': int(outcome[5].split(":")[0]), 'g_conc': int(outcome[5].split(":")[1]), 'g_diff': int(outcome[6]),
                    'pts': float(outcome[7].split(":")[0]),
                    'rank': int(outcome[8].replace("<b>", "").replace("</b>", ""))}
        except:
            pass

    results[id] = team_results

    return results

In [170]:
season = '09/10'
id = 29
scrape_seasonal_performance(id).get(id, {season: 0})[season]

{'tier': 'Primo livello',
 'wins': 16,
 'draws': 13,
 'loss': 9,
 'g_scored': 60,
 'g_conc': 49,
 'g_diff': 11,
 'pts': 61.0,
 'rank': 8}

### APPLY

**First 10 rows**

In [15]:
results = scrape_seasonal_performance(985)
results[985].get('50/51')

{'tier': 'Primo livello',
 'wins': 24,
 'draws': 8,
 'loss': 10,
 'g_scored': 74,
 'g_conc': 40,
 'g_diff': 34,
 'pts': 56.0,
 'rank': 2}

In [20]:
df_temp = df[:10]

ids = df_temp["team_id"].tolist()
seasons = df_temp.columns[2:]

results = {}
for id in tqdm(ids):
    results.update(scrape_seasonal_performance(id))
    #print(results)

for season in seasons:
    df_temp[season] = df_temp['team_id'].map(lambda x: results[x].get(season, 0))

df_temp

100%|██████████| 10/10 [00:30<00:00,  3.06s/it]
C:\Users\feder\AppData\Local\Temp\ipykernel_10936\3281108007.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp[season] = df_temp['team_id'].map(lambda x: results[x].get(season, 0))


,team_id,team_name,09/10,10/11,11/12,12/13,13/14,14/15,15/16,16/17,17/18,18/19,19/20,20/21,21/22,22/23,23/24
0,985,Manchester United,"{'tier': 'Primo livello', 'wins': 27, 'draws':...","{'tier': 'Primo livello', 'wins': 23, 'draws':...","{'tier': 'Primo livello', 'wins': 28, 'draws':...","{'tier': 'Primo livello', 'wins': 28, 'draws':...","{'tier': 'Primo livello', 'wins': 19, 'draws':...","{'tier': 'Primo livello', 'wins': 20, 'draws':...","{'tier': 'Primo livello', 'wins': 19, 'draws':...","{'tier': 'Primo livello', 'wins': 18, 'draws':...","{'tier': 'Primo livello', 'wins': 25, 'draws':...","{'tier': 'Primo livello', 'wins': 19, 'draws':...","{'tier': 'Primo livello', 'wins': 18, 'draws':...","{'tier': 'Primo livello', 'wins': 21, 'draws':...","{'tier': 'Primo livello', 'wins': 16, 'draws':...","{'tier': 'Primo livello', 'wins': 23, 'draws':...","{'tier': 'Primo livello', 'wins': 17, 'draws':..."
26,31,Liverpool FC,"{'tier': 'Primo livello', 'wins': 18, 'draws':...","{'tier': 'Primo livello', 'wins': 17, 'draws':...","{'tier': 'Primo livello', 'wins': 14, 'draws':...","{'tier': 'Primo livello', 'wins': 16, 'draws':...","{'tier': 'Primo livello', 'wins': 26, 'draws':...","{'tier': 'Primo livello', 'wins': 18, 'draws':...","{'tier': 'Primo livello', 'wins': 16, 'draws':...","{'tier': 'Primo livello', 'wins': 22, 'draws':...","{'tier': 'Primo livello', 'wins': 21, 'draws':...","{'tier': 'Primo livello', 'wins': 30, 'draws':...","{'tier': 'Primo livello', 'wins': 32, 'draws':...","{'tier': 'Primo livello', 'wins': 20, 'draws':...","{'tier': 'Primo livello', 'wins': 28, 'draws':...","{'tier': 'Primo livello', 'wins': 19, 'draws':...","{'tier': 'Primo livello', 'wins': 23, 'draws':..."
57,631,Chelsea FC,"{'tier': 'Primo livello', 'wins': 27, 'draws':...","{'tier': 'Primo livello', 'wins': 21, 'draws':...","{'tier': 'Primo livello', 'wins': 18, 'draws':...","{'tier': 'Primo livello', 'wins': 22, 'draws':...","{'tier': 'Primo livello', 'wins': 25, 'draws':...","{'tier': 'Primo livello', 'wins': 26, 'draws':...","{'tier': 'Primo livello', 'wins': 12, 'draws':...","{'tier': 'Primo livello', 'wins': 30, 'draws':...","{'tier': 'Primo livello', 'wins': 21, 'draws':...","{'tier': 'Primo livello', 'wins': 21, 'draws':...","{'tier': 'Primo livello', 'wins': 20, 'draws':...","{'tier': 'Primo livello', 'wins': 19, 'draws':...","{'tier': 'Primo livello', 'wins': 21, 'draws':...","{'tier': 'Primo livello', 'wins': 11, 'draws':...","{'tier': 'Primo livello', 'wins': 17, 'draws':..."
83,11,Arsenal FC,"{'tier': 'Primo livello', 'wins': 23, 'draws':...","{'tier': 'Primo livello', 'wins': 19, 'draws':...","{'tier': 'Primo livello', 'wins': 21, 'draws':...","{'tier': 'Primo livello', 'wins': 21, 'draws':...","{'tier': 'Primo livello', 'wins': 24, 'draws':...","{'tier': 'Primo livello', 'wins': 22, 'draws':...","{'tier': 'Primo livello', 'wins': 20, 'draws':...","{'tier': 'Primo livello', 'wins': 23, 'draws':...","{'tier': 'Primo livello', 'wins': 19, 'draws':...","{'tier': 'Primo livello', 'wins': 21, 'draws':...","{'tier': 'Primo livello', 'wins': 14, 'draws':...","{'tier': 'Primo livello', 'wins': 18, 'draws':...","{'tier': 'Primo livello', 'wins': 22, 'draws':...","{'tier': 'Primo livello', 'wins': 26, 'draws':...","{'tier': 'Primo livello', 'wins': 27, 'draws':..."
103,29,Everton FC,"{'tier': 'Primo livello', 'wins': 16, 'draws':...","{'tier': 'Primo livello', 'wins': 13, 'draws':...","{'tier': 'Primo livello', 'wins': 15, 'draws':...","{'tier': 'Primo livello', 'wins': 16, 'draws':...","{'tier': 'Primo livello', 'wins': 21, 'draws':...","{'tier': 'Primo livello', 'wins': 12, 'draws':...","{'tier': 'Primo livello', 'wins': 11, 'draws':...","{'tier': 'Primo livello', 'wins': 17, 'draws':...","{'tier': 'Primo livello', 'wins': 13, 'draws':...","{'tier': 'Primo livello', 'wins': 15, 'draws':...","{'tier': 'Primo livello', 'wins': 13, 'draws':...","{'tier': 'Primo livello', 'wins': 17, 'draws':...","{'tier': 'Primo livello', 'wins': 11, 'draws':..."

In [ ]:
# OLD
df_temp = df[:10]
for season in tqdm(df.columns[2:]):
    df_temp[season] = df_temp['team_id'].map(lambda x: scrape_seasonal_performance(x).get(x, {season: 0})[season])

**All**

In [5]:
# Lista degli ID dei team e stagioni
ids = df["team_id"].tolist()
seasons = df.columns[2:]

# Funzione di scraping in parallelo
with concurrent.futures.ThreadPoolExecutor() as executor:
    results_list = list(tqdm(executor.map(scrape_seasonal_performance, ids), total=len(ids)))


100%|██████████| 257/257 [02:10<00:00,  1.98it/s]


In [6]:
results = {}
for result in results_list:
    results.update(result)

# Aggiorna il DataFrame con i risultati ottenuti
for season in seasons:
    df[season] = df['team_id'].map(lambda x: results[x].get(season, 0))



In [8]:
# Visualizza il DataFrame aggiornato
df.head(10)

,team_id,team_name,09/10,10/11,11/12,12/13,13/14,14/15,15/16,16/17,17/18,18/19,19/20,20/21,21/22,22/23,23/24
0,985,Manchester United,"{'tier': 'Primo livello', 'wins': 27, 'draws':...","{'tier': 'Primo livello', 'wins': 23, 'draws':...","{'tier': 'Primo livello', 'wins': 28, 'draws':...","{'tier': 'Primo livello', 'wins': 28, 'draws':...","{'tier': 'Primo livello', 'wins': 19, 'draws':...","{'tier': 'Primo livello', 'wins': 20, 'draws':...","{'tier': 'Primo livello', 'wins': 19, 'draws':...","{'tier': 'Primo livello', 'wins': 18, 'draws':...","{'tier': 'Primo livello', 'wins': 25, 'draws':...","{'tier': 'Primo livello', 'wins': 19, 'draws':...","{'tier': 'Primo livello', 'wins': 18, 'draws':...","{'tier': 'Primo livello', 'wins': 21, 'draws':...","{'tier': 'Primo livello', 'wins': 16, 'draws':...","{'tier': 'Primo livello', 'wins': 23, 'draws':...","{'tier': 'Primo livello', 'wins': 17, 'draws':..."
26,31,Liverpool FC,"{'tier': 'Primo livello', 'wins': 18, 'draws':...","{'tier': 'Primo livello', 'wins': 17, 'draws':...","{'tier': 'Primo livello', 'wins': 14, 'draws':...","{'tier': 'Primo livello', 'wins': 16, 'draws':...","{'tier': 'Primo livello', 'wins': 26, 'draws':...","{'tier': 'Primo livello', 'wins': 18, 'draws':...","{'tier': 'Primo livello', 'wins': 16, 'draws':...","{'tier': 'Primo livello', 'wins': 22, 'draws':...","{'tier': 'Primo livello', 'wins': 21, 'draws':...","{'tier': 'Primo livello', 'wins': 30, 'draws':...","{'tier': 'Primo livello', 'wins': 32, 'draws':...","{'tier': 'Primo livello', 'wins': 20, 'draws':...","{'tier': 'Primo livello', 'wins': 28, 'draws':...","{'tier': 'Primo livello', 'wins': 19, 'draws':...","{'tier': 'Primo livello', 'wins': 23, 'draws':..."
57,631,Chelsea FC,"{'tier': 'Primo livello', 'wins': 27, 'draws':...","{'tier': 'Primo livello', 'wins': 21, 'draws':...","{'tier': 'Primo livello', 'wins': 18, 'draws':...","{'tier': 'Primo livello', 'wins': 22, 'draws':...","{'tier': 'Primo livello', 'wins': 25, 'draws':...","{'tier': 'Primo livello', 'wins': 26, 'draws':...","{'tier': 'Primo livello', 'wins': 12, 'draws':...","{'tier': 'Primo livello', 'wins': 30, 'draws':...","{'tier': 'Primo livello', 'wins': 21, 'draws':...","{'tier': 'Primo livello', 'wins': 21, 'draws':...","{'tier': 'Primo livello', 'wins': 20, 'draws':...","{'tier': 'Primo livello', 'wins': 19, 'draws':...","{'tier': 'Primo livello', 'wins': 21, 'draws':...","{'tier': 'Primo livello', 'wins': 11, 'draws':...","{'tier': 'Primo livello', 'wins': 17, 'draws':..."
83,11,Arsenal FC,"{'tier': 'Primo livello', 'wins': 23, 'draws':...","{'tier': 'Primo livello', 'wins': 19, 'draws':...","{'tier': 'Primo livello', 'wins': 21, 'draws':...","{'tier': 'Primo livello', 'wins': 21, 'draws':...","{'tier': 'Primo livello', 'wins': 24, 'draws':...","{'tier': 'Primo livello', 'wins': 22, 'draws':...","{'tier': 'Primo livello', 'wins': 20, 'draws':...","{'tier': 'Primo livello', 'wins': 23, 'draws':...","{'tier': 'Primo livello', 'wins': 19, 'draws':...","{'tier': 'Primo livello', 'wins': 21, 'draws':...","{'tier': 'Primo livello', 'wins': 14, 'draws':...","{'tier': 'Primo livello', 'wins': 18, 'draws':...","{'tier': 'Primo livello', 'wins': 22, 'draws':...","{'tier': 'Primo livello', 'wins': 26, 'draws':...","{'tier': 'Primo livello', 'wins': 27, 'draws':..."
103,29,Everton FC,"{'tier': 'Primo livello', 'wins': 16, 'draws':...","{'tier': 'Primo livello', 'wins': 13, 'draws':...","{'tier': 'Primo livello', 'wins': 15, 'draws':...","{'tier': 'Primo livello', 'wins': 16, 'draws':...","{'tier': 'Primo livello', 'wins': 21, 'draws':...","{'tier': 'Primo livello', 'wins': 12, 'draws':...","{'tier': 'Primo livello', 'wins': 11, 'draws':...","{'tier': 'Primo livello', 'wins': 17, 'draws':...","{'tier': 'Primo livello', 'wins': 13, 'draws':...","{'tier': 'Primo livello', 'wins': 15, 'draws':...","{'tier': 'Primo livello', 'wins': 13, 'draws':...","{'tier': 'Primo livello', 'wins': 17, 'draws':...","{'tier': 'Primo livello', 'wins': 11, 'draws':..."

In [9]:
df.to_csv("dataset\champ_performances.csv")